In [ ]:
# Task 1 
# Rotate Left Function
def rotl(x: int, n: int = 1) -> int:
    """
    Rotates the bits of a 32-bit unsigned integer to the left by n positions.
    """
    n = n % 32  # Keep n in the range 0-31
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF
